In [1]:
import pandas as pd # importing pandas
import numpy as np # importing numpy
import matplotlib.pyplot as plt  # importing matplotlib
from sklearn.decomposition import KernelPCA #importing kernelpca
from sklearn.metrics import r2_score #importing r2 score
from sklearn.metrics import mean_squared_error

movie=pd.read_csv('ml360datasets/Movie_classification.csv') #reading the movie classification data
house=pd.read_csv('C:/Users/Administrator/Downloads/ml360datasets/kc_house.csv') # reading the house data
loan=pd.read_csv('C:/Users/Administrator/Downloads/ml360datasets/loan_classification.csv') #reading the loan classification
wine=pd.read_csv('C:/Users/Administrator/Downloads/ml360datasets/winequality-white.csv') #reading the wine data
iris=pd.read_csv('C:/Users/Administrator/Downloads/ml360datasets/Iris.csv') #reading the iris data
insurance=pd.read_csv('C:/Users/Administrator/Downloads/ml360datasets/insurance.csv') #reading the iris data

def null_replace(df): #define fun for replacing the null values
    for col in df.columns:
        if (df[col].dtypes == "object"): # checking the datatype
            df[col]=df[col].fillna(df[col].mode()[0]) #replacing the null values for catgeory column
        else:
            df[col]=df[col].fillna(df[col].mean()) # replacing the null values for numerical columns
    return df

def label_encoder(df): #define fun for label encoder 
    from sklearn.preprocessing import LabelEncoder #doing the labelencoder for category columns
    le = LabelEncoder() # giving variable for labelencoder
    for col in df.columns:  
        if (df[col].dtypes == "object"): #checking the data type of the column
            df[col] = df[col].astype(str) 
            df[col] = le.fit_transform(df[col]) #applying the label encoder
    return df #returns dataframe 


house=null_replace(house) #appying the function for replacing null values
loan=null_replace(loan)  #appying the function for replacing null values
wine=null_replace(wine)  #appying the function for replacing null values

# In kernel pca we trying to optimize N components , kernel and Gamma values 

In [ ]:
#N_components we are trying to identify by the cummulative varience explained. we are taking nearest value of 95(which occupies
#max variance). the nearest value of 95 variance that index point we are considering it has N_components


#In Kernel we have different types of them to decide we are calculatng the reconstruction error of Original and inverse 
#transfrommed features the difference between them which ever kernel is low will choose that kernel

#In Gamma value only specific kerenl will consider ['poly','rbf', 'sigmoid'] for these kernel the optimal value is 
#1 / (len(X.columns) * X.values.var())

#for remaining kernel they will ignore the gamma value in that case the default gamma value will be 1/len(X.columns)


In [ ]:
#As per my observation The kernel Pca is working for classification data not on regression data 
#for classification I took Movie, Loan , Iris,wine
#for Regression Wine, house, insurance

# Movie

In [2]:
movie=null_replace(movie) #appying the function for replacing null values
movie=label_encoder(movie) #appying the function for label encoder

#------------------------------------------------------------------------------------------------------------------------------#
X=movie.drop('Start_Tech_Oscar',axis=1) # storing the X matrix 
y=movie['Start_Tech_Oscar'] # defing the y as target variable

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

X_covariance_matrix = np.cov(X_train.T) #calculating the covariance for Transformed X matrix 

eig_vals, eig_vecs = np.linalg.eig(X_covariance_matrix) #calculating the eigenvales and eigen vectors

# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort(key=lambda x: x[0], reverse=True)


tot = sum(eig_vals) #calculating the sum of eigen values
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)] #soring the eigenvalues and calculating the variance explained
cum_var_exp = np.cumsum(var_exp) #calculating the cumulative sum  of variance explained 
print ("Variance captured by each component is \n",var_exp)
print(40 * '-')
print ("Cumulative variance captured as we travel each component \n",cum_var_exp)
print('\n')


value=min(cum_var_exp, key=lambda x:abs(x-95)) # identifing the neareast value of 95 in the list 

n_components=list(cum_var_exp).index(value) # getting that nearest value of 95 and storing the index of that value in list
#as no.of components


from numpy import linalg as LA # importing the LA
kernel=['linear','poly','rbf', 'sigmoid', 'cosine']  #giving the list of kernels in Kernelpca
reconstruction_error=[] #empty list of reconstruction error
for i in  kernel:
    if kernel == ['poly','rbf', 'sigmoid']: #if these kernel's are comes then this gamma value has to choose 
        gamma=1 / (len(X.columns) * X.values.var())
    else:
        gamma=1/len(X.columns) #else the gamma values will be this 
    pca = KernelPCA(n_components=n_components,kernel=i,gamma=gamma,fit_inverse_transform=True)# deffing the kernelpca and 
    #the important parameters 
    pca2_results = pca.fit_transform(X_train) #fitting kernel pca 
    pca2_proj_back=pca.inverse_transform(pca2_results) #performing the inverse transform on the transformed fetaures with kernelpca
    reconstruction_error.append(LA.norm((X_train, pca2_proj_back),None)) #calculating the difference between the orginal 
    #feature and inverse transformed features of kernel pca
    
print('reconstruction_error',reconstruction_error) #printing the reconstruction error
print('\n')



#------------------------------------------------------------------------------------------------------------------------------#
X=movie.drop('Start_Tech_Oscar',axis=1) #deffing the X features 
y=movie['Start_Tech_Oscar'] #deffing the target variable 

from sklearn.preprocessing import StandardScaler #importing the standard scaler
sc = StandardScaler() #defing the standard scaler
X_train = sc.fit_transform(X) #ftting and transfroming the X features


pca = KernelPCA(n_components=n_components,kernel='linear',gamma=1/len(X.columns)) # defing the kernelpca parameters
# checking kernel parameter with linear and respective gamma value
pca2_results = pca.fit_transform(X_train) #fitting and transforming the X features

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestClassifier  #importing the randomforest

classifier= RandomForestClassifier() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import classification_report #importing the classification report
print('KernelPca Linear Model Results','\n',classification_report(y_test, y_pred)) #printing the classification report
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=movie.drop('Start_Tech_Oscar',axis=1) #storing the X features 
y=movie['Start_Tech_Oscar'] # defining the target variable

from sklearn.preprocessing import StandardScaler #importing the standard scaler 
sc = StandardScaler()  #defining the standardscaler
X_train = sc.fit_transform(X) #fit and transforming the X features 

pca = KernelPCA(n_components=n_components,kernel='poly',gamma=1 / (len(X.columns) * X.values.var()))# defing the kernelpca parameters
# checking kernel parameter with poly and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestClassifier  #importing the randomforest

classifier= RandomForestClassifier() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import classification_report #importing the classification report
print('KernelPca poly Model Results','\n',classification_report(y_test, y_pred)) #printing the classification report
print('\n')


#------------------------------------------------------------------------------------------------------------------------------#
X=movie.drop('Start_Tech_Oscar',axis=1) #storing the X features 
y=movie['Start_Tech_Oscar'] # defining the target variable

from sklearn.preprocessing import StandardScaler #importing the standard scaler 
sc = StandardScaler()  #defining the standardscaler
X_train = sc.fit_transform(X) #fit and transforming the X features 

pca = KernelPCA(n_components=n_components,kernel='rbf',gamma=1 / (len(X.columns) * X.values.var()))# defing the kernelpca parameters
# checking kernel parameter with rbf and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestClassifier  #importing the randomforest

classifier= RandomForestClassifier() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import classification_report #importing the classification report
print('KernelPca rbf Model Results','\n',classification_report(y_test, y_pred)) #printing the classification report
print('\n')


#------------------------------------------------------------------------------------------------------------------------------#
X=movie.drop('Start_Tech_Oscar',axis=1) #storing the X features 
y=movie['Start_Tech_Oscar'] # defining the target variable

from sklearn.preprocessing import StandardScaler #importing the standard scaler 
sc = StandardScaler()  #defining the standardscaler
X_train = sc.fit_transform(X) #fit and transforming the X features 

pca = KernelPCA(n_components=n_components,kernel='sigmoid',gamma=1 / (len(X.columns) * X.values.var()))# defing the kernelpca parameters
# checking kernel parameter with sigmoid and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestClassifier  #importing the randomforest

classifier= RandomForestClassifier() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import classification_report #importing the classification report
print('KernelPca sigmoid Model Results','\n',classification_report(y_test, y_pred)) #printing the classification report
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=movie.drop('Start_Tech_Oscar',axis=1) #storing the X features 
y=movie['Start_Tech_Oscar'] # defining the target variable

from sklearn.preprocessing import StandardScaler #importing the standard scaler 
sc = StandardScaler()  #defining the standardscaler
X_train = sc.fit_transform(X) #fit and transforming the X features 

pca = KernelPCA(n_components=n_components,kernel='cosine',gamma=1 / (len(X.columns)))# defing the kernelpca parameters
# checking kernel parameter with sigmoid and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestClassifier  #importing the randomforest

classifier= RandomForestClassifier() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import classification_report #importing the classification report
print('KernelPca cosine Model Results','\n',classification_report(y_test, y_pred)) #printing the classification report
print('\n')

#we can check the reconstruction error is less for sigmoid and we got better results for sigmoid kernel

Variance captured by each component is 
 [42.187448758298764, 11.255143435368113, 6.517298747176463, 5.8258940686909835, 5.451824486345361, 5.290035849717573, 5.074730219686693, 4.419500183710625, 4.136709602334205, 3.101133602095518, 2.1894937006237005, 1.5637588131952458, 1.3270823433819776, 1.1810573027811653, 0.4135391232374326, 0.04281361642512702, 0.012224637176785525, 0.010311509754266958]
----------------------------------------
Cumulative variance captured as we travel each component 
 [ 42.18744876  53.44259219  59.95989094  65.78578501  71.2376095
  76.52764535  81.60237557  86.02187575  90.15858535  93.25971895
  95.44921265  97.01297147  98.34005381  99.52111111  99.93465024
  99.97746385  99.98968849 100.        ]


reconstruction_error [132.59771577319376, 126.67090213246853, 113.79765444432853, 105.76283782688972, 128.02851394690126]


KernelPca Linear Model Results 
               precision    recall  f1-score   support

           0       0.52      0.46      0.49     

# Loan

In [3]:
loan=null_replace(loan) #appying the function for replacing null values
loan=label_encoder(loan) #appying the function for label encoder


#------------------------------------------------------------------------------------------------------------------------------#
X=loan.drop('Loan_Status',axis=1) #storing the X features 
y=loan['Loan_Status'] # defining the target variable


from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

X_covariance_matrix = np.cov(X_train.T) #calculating the covariance for Transformed X matrix 

eig_vals, eig_vecs = np.linalg.eig(X_covariance_matrix) #calculating the eigenvales and eigen vectors

# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort(key=lambda x: x[0], reverse=True)


tot = sum(eig_vals) #calculating the sum of eigen values
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)] #soring the eigenvalues and calculating the variance explained
cum_var_exp = np.cumsum(var_exp) #calculating the cumulative sum  of variance explained 
print ("Variance captured by each component is \n",var_exp)
print(40 * '-')
print ("Cumulative variance captured as we travel each component \n",cum_var_exp)
print('\n')


value=min(cum_var_exp, key=lambda x:abs(x-95)) # identifing the neareast value of 95 in the list 

n_components=list(cum_var_exp).index(value) # getting that nearest value of 95 and storing the index of that value in list
#as no.of components


from numpy import linalg as LA # importing the LA
kernel=['linear','poly','rbf', 'sigmoid', 'cosine']  #giving the list of kernels in Kernelpca
reconstruction_error=[] #empty list of reconstruction error
for i in  kernel:
    if kernel == ['poly','rbf', 'sigmoid']: #if these kernel's are comes then this gamma value has to choose 
        gamma=1 / (len(X.columns) * X.values.var())
    else:
        gamma=1/len(X.columns) #else the gamma values will be this 
    pca = KernelPCA(n_components=n_components,kernel=i,gamma=gamma,fit_inverse_transform=True)# deffing the kernelpca and 
    #the important parameters 
    pca2_results = pca.fit_transform(X_train) #fitting kernel pca 
    pca2_proj_back=pca.inverse_transform(pca2_results) #performing the inverse transform on the transformed fetaures with kernelpca
    reconstruction_error.append(LA.norm((X_train, pca2_proj_back),None)) #calculating the difference between the orginal 
    #feature and inverse transformed features of kernel pca
    
print('reconstruction_error',reconstruction_error) #printing the reconstruction error
print('\n')



#------------------------------------------------------------------------------------------------------------------------------#
X=loan.drop('Loan_Status',axis=1) #storing the X features 
y=loan['Loan_Status'] # defining the target variable


from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 


pca = KernelPCA(n_components=n_components,kernel='linear',gamma=1/len(X.columns)) # defing the kernelpca parameters
# checking kernel parameter with linear and respective gamma value
pca2_results = pca.fit_transform(X_train) #fitting and transforming the X features

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestClassifier  #importing the randomforest

classifier= RandomForestClassifier() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import classification_report #importing the classification report
print('KernelPca Linear Model Results','\n',classification_report(y_test, y_pred)) #printing the classification report
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=loan.drop('Loan_Status',axis=1) #storing the X features 
y=loan['Loan_Status'] # defining the target variable


from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='poly',gamma=1 / (len(X.columns) * X.values.var()))# defing the kernelpca parameters
# checking kernel parameter with poly and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestClassifier  #importing the randomforest

classifier= RandomForestClassifier() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import classification_report #importing the classification report
print('KernelPca poly Model Results','\n',classification_report(y_test, y_pred)) #printing the classification report
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=loan.drop('Loan_Status',axis=1) #storing the X features 
y=loan['Loan_Status'] # defining the target variable


from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='rbf',gamma=1 / (len(X.columns) * X.values.var())) # defing the kernelpca parameters
# checking kernel parameter with rbf and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestClassifier  #importing the randomforest

classifier= RandomForestClassifier() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import classification_report #importing the classification report
print('KernelPca rbf Model Results','\n',classification_report(y_test, y_pred)) #printing the classification report
print('\n')


#------------------------------------------------------------------------------------------------------------------------------#
X=loan.drop('Loan_Status',axis=1) #storing the X features 
y=loan['Loan_Status'] # defining the target variable


from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='sigmoid',gamma=1 / (len(X.columns) * X.values.var()))# defing the kernelpca parameters
# checking kernel parameter with sigmoid and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestClassifier  #importing the randomforest

classifier= RandomForestClassifier() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import classification_report #importing the classification report
print('KernelPca Sigmoid Model Results','\n',classification_report(y_test, y_pred)) #printing the classification report
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=loan.drop('Loan_Status',axis=1) #storing the X features 
y=loan['Loan_Status'] # defining the target variable


from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='cosine',gamma=1 / (len(X.columns)))# defing the kernelpca parameters
# checking kernel parameter with cosine and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestClassifier  #importing the randomforest

classifier= RandomForestClassifier() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import classification_report #importing the classification report
print('KernelPca Cosine Model Results','\n',classification_report(y_test, y_pred)) #printing the classification report
print('\n')

Variance captured by each component is 
 [15.623882996887438, 12.229427600969263, 10.114824810762203, 9.252034660414994, 8.586944365899377, 8.324837184372615, 7.654349575594069, 7.072443596299781, 6.929544893743198, 6.469504079041541, 4.847404575367141, 2.8948016606483766]
----------------------------------------
Cumulative variance captured as we travel each component 
 [ 15.623883    27.8533106   37.96813541  47.22017007  55.80711443
  64.13195162  71.78630119  78.85874479  85.78828968  92.25779376
  97.10519834 100.        ]


reconstruction_error [118.93528231946378, 110.71571835329996, 102.71549023290447, 92.84844034936181, 114.50515757751825]


KernelPca Linear Model Results 
               precision    recall  f1-score   support

           0       0.80      0.51      0.62        65
           1       0.78      0.93      0.85       120

    accuracy                           0.78       185
   macro avg       0.79      0.72      0.74       185
weighted avg       0.79      0.78   

# Iris

In [5]:
iris=null_replace(iris)
iris=label_encoder(iris)

iris=null_replace(iris) #appying the function for replacing null values
iris=label_encoder(iris) #appying the function for label encoder


#------------------------------------------------------------------------------------------------------------------------------#
X=iris.drop('Species',axis=1)  #storing the X features 
y=iris['Species'] # defining the target variable


from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

X_covariance_matrix = np.cov(X_train.T) #calculating the covariance for Transformed X matrix 

eig_vals, eig_vecs = np.linalg.eig(X_covariance_matrix) #calculating the eigenvales and eigen vectors

# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort(key=lambda x: x[0], reverse=True)


tot = sum(eig_vals) #calculating the sum of eigen values
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)] #soring the eigenvalues and calculating the variance explained
cum_var_exp = np.cumsum(var_exp) #calculating the cumulative sum  of variance explained 
print ("Variance captured by each component is \n",var_exp)
print(40 * '-')
print ("Cumulative variance captured as we travel each component \n",cum_var_exp)
print('\n')


value=min(cum_var_exp, key=lambda x:abs(x-95)) # identifing the neareast value of 95 in the list 

n_components=list(cum_var_exp).index(value) # getting that nearest value of 95 and storing the index of that value in list
#as no.of components


from numpy import linalg as LA # importing the LA
kernel=['linear','poly','rbf', 'sigmoid', 'cosine']  #giving the list of kernels in Kernelpca
reconstruction_error=[] #empty list of reconstruction error
for i in  kernel:
    if kernel == ['poly','rbf', 'sigmoid']: #if these kernel's are comes then this gamma value has to choose 
        gamma=1 / (len(X.columns) * X.values.var())
    else:
        gamma=1/len(X.columns) #else the gamma values will be this 
    pca = KernelPCA(n_components=n_components,kernel=i,gamma=gamma,fit_inverse_transform=True)# deffing the kernelpca and 
    #the important parameters 
    pca2_results = pca.fit_transform(X_train) #fitting kernel pca 
    pca2_proj_back=pca.inverse_transform(pca2_results) #performing the inverse transform on the transformed fetaures with kernelpca
    reconstruction_error.append(LA.norm((X_train, pca2_proj_back),None)) #calculating the difference between the orginal 
    #feature and inverse transformed features of kernel pca
    
print('reconstruction_error',reconstruction_error) #printing the reconstruction error
print('\n')


#------------------------------------------------------------------------------------------------------------------------------#
X=iris.drop('Species',axis=1)  #storing the X features 
y=iris['Species'] # defining the target variable


from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='linear',gamma=1/len(X.columns)) # defing the kernelpca parameters
# checking kernel parameter with linear and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestClassifier  #importing the randomforest

classifier= RandomForestClassifier() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import classification_report #importing the classification report
print('KernelPca Linear Model Results','\n',classification_report(y_test, y_pred)) #printing the classification report
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=iris.drop('Species',axis=1)
y=iris['Species']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='poly',gamma=1 / (len(X.columns) * X.values.var()))# defing the kernelpca parameters
# checking kernel parameter with poly and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestClassifier  #importing the randomforest

classifier= RandomForestClassifier() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import classification_report #importing the classification report
print('KernelPca poly Model Results','\n',classification_report(y_test, y_pred)) #printing the classification report
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=iris.drop('Species',axis=1)
y=iris['Species']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='rbf',gamma=1 / (len(X.columns) * X.values.var())) # defing the kernelpca parameters
# checking kernel parameter with rbf and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestClassifier  #importing the randomforest

classifier= RandomForestClassifier() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import classification_report #importing the classification report
print('KernelPca rbf Model Results','\n',classification_report(y_test, y_pred)) #printing the classification report
print('\n')


#------------------------------------------------------------------------------------------------------------------------------#
X=iris.drop('Species',axis=1)
y=iris['Species']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='sigmoid',gamma=1 / (len(X.columns) * X.values.var()))# defing the kernelpca parameters
# checking kernel parameter with sigmoid and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestClassifier  #importing the randomforest

classifier= RandomForestClassifier() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import classification_report #importing the classification report
print('KernelPca Sigmoid Model Results','\n',classification_report(y_test, y_pred)) #printing the classification report
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=iris.drop('Species',axis=1)
y=iris['Species']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='cosine',gamma=1 / (len(X.columns)))# defing the kernelpca parameters
# checking kernel parameter with cosine and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestClassifier  #importing the randomforest

classifier= RandomForestClassifier() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 


from sklearn.metrics import classification_report #importing the classification report
print('KernelPca cosine Model Results','\n',classification_report(y_test, y_pred))#printing the classification report
print('\n')

Variance captured by each component is 
 [74.70533000277261, 18.435256887931974, 4.682624420508823, 1.7647673742565004, 0.41202131453009]
----------------------------------------
Cumulative variance captured as we travel each component 
 [ 74.70533     93.14058689  97.82321131  99.58797869 100.        ]


reconstruction_error [36.17036517202373, 36.89625093647699, 34.27594225580259, 33.46060298114652, 34.2012999236542]


KernelPca Linear Model Results 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      0.92      0.96        13
           2       0.93      1.00      0.96        13

    accuracy                           0.98        45
   macro avg       0.98      0.97      0.97        45
weighted avg       0.98      0.98      0.98        45



KernelPca poly Model Results 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       

# Wine - Classification

In [11]:
wine=null_replace(wine) #appying the function for replacing null values
wine=label_encoder(wine) #appying the function for label encoder


#------------------------------------------------------------------------------------------------------------------------------#
X=wine.drop('quality',axis=1)  #storing the X features 
y=wine['quality'] # defining the target variable


from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

X_covariance_matrix = np.cov(X_train.T) #calculating the covariance for Transformed X matrix 

eig_vals, eig_vecs = np.linalg.eig(X_covariance_matrix) #calculating the eigenvales and eigen vectors

# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort(key=lambda x: x[0], reverse=True)


tot = sum(eig_vals) #calculating the sum of eigen values
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)] #soring the eigenvalues and calculating the variance explained
cum_var_exp = np.cumsum(var_exp) #calculating the cumulative sum  of variance explained 
print ("Variance captured by each component is \n",var_exp)
print(40 * '-')
print ("Cumulative variance captured as we travel each component \n",cum_var_exp)
print('\n')


value=min(cum_var_exp, key=lambda x:abs(x-95)) # identifing the neareast value of 95 in the list 

n_components=list(cum_var_exp).index(value) # getting that nearest value of 95 and storing the index of that value in list
#as no.of components


from numpy import linalg as LA # importing the LA
kernel=['linear','poly','rbf', 'sigmoid', 'cosine']  #giving the list of kernels in Kernelpca
reconstruction_error=[] #empty list of reconstruction error
for i in  kernel:
    if kernel == ['poly','rbf', 'sigmoid']: #if these kernel's are comes then this gamma value has to choose 
        gamma=1 / (len(X.columns) * X.values.var())
    else:
        gamma=1/len(X.columns) #else the gamma values will be this 
    pca = KernelPCA(n_components=n_components,kernel=i,gamma=gamma,fit_inverse_transform=True)# deffing the kernelpca and 
    #the important parameters 
    pca2_results = pca.fit_transform(X_train) #fitting kernel pca 
    pca2_proj_back=pca.inverse_transform(pca2_results) #performing the inverse transform on the transformed fetaures with kernelpca
    reconstruction_error.append(LA.norm((X_train, pca2_proj_back),None)) #calculating the difference between the orginal 
    #feature and inverse transformed features of kernel pca
    
print('reconstruction_error',reconstruction_error) #printing the reconstruction error
print('\n')


#------------------------------------------------------------------------------------------------------------------------------#
X=wine.drop('quality',axis=1)  #storing the X features 
y=wine['quality'] # defining the target variable


from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='linear',gamma=1/len(X.columns)) # defing the kernelpca parameters
# checking kernel parameter with linear and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestClassifier   #importing the randomforest

classifier= RandomForestClassifier () # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 


from sklearn.metrics import  classification_report  #importing the classification_report
print('KernelPca Linear Model Results','\n', classification_report(y_test, y_pred))#printing the classification_report
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=wine.drop('quality',axis=1)
y=wine['quality']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='poly',gamma=1 / (len(X.columns) * X.values.var()))# defing the kernelpca parameters
# checking kernel parameter with poly and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestClassifier   #importing the randomforest

classifier= RandomForestClassifier () # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import  classification_report
print('KernelPca poly Model Results','\n', classification_report(y_test, y_pred))
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=wine.drop('quality',axis=1)
y=wine['quality']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='rbf',gamma=1 / (len(X.columns) * X.values.var())) # defing the kernelpca parameters
# checking kernel parameter with rbf and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestClassifier   #importing the randomforest

classifier= RandomForestClassifier () # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import  classification_report
print('KernelPca rbf Model Results','\n', classification_report(y_test, y_pred))
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=wine.drop('quality',axis=1)
y=wine['quality']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='sigmoid',gamma=1 / (len(X.columns) * X.values.var()))# defing the kernelpca parameters
# checking kernel parameter with sigmoid and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestClassifier   #importing the randomforest

classifier= RandomForestClassifier () # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import  classification_report
print('KernelPca sigmoid Model Results','\n', classification_report(y_test, y_pred))
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=wine.drop('quality',axis=1)
y=wine['quality']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='cosine',gamma=1 / (len(X.columns)))# defing the kernelpca parameters
# checking kernel parameter with cosine and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestClassifier   #importing the randomforest

classifier= RandomForestClassifier () # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 


from sklearn.metrics import  classification_report
print('KernelPca cosine Model Results','\n', classification_report(y_test, y_pred))

Variance captured by each component is 
 [29.29321718754935, 14.320363009824618, 11.106103089574539, 9.259294133372196, 8.84849617879475, 8.534013740313142, 6.605436572268536, 5.448713451289449, 3.7649424158637173, 2.6317012597137124, 0.18771896143598701]
----------------------------------------
Cumulative variance captured as we travel each component 
 [ 29.29321719  43.6135802   54.71968329  63.97897742  72.8274736
  81.36148734  87.96692391  93.41563736  97.18057978  99.81228104
 100.        ]


reconstruction_error [318.21191534955653, 306.298271654715, 295.49157949581024, 303.89915616908604, 308.36087301471014]


KernelPca Linear Model Results 
               precision    recall  f1-score   support

           3       0.50      0.14      0.22         7
           4       0.50      0.15      0.23        40
           5       0.68      0.70      0.69       426
           6       0.66      0.77      0.71       668
           7       0.68      0.52      0.59       280
           8    

C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


-------------------------------------------------------------------------------------------------------------------------------

# Wine Regression

In [7]:
wine=null_replace(wine) #appying the function for replacing null values
wine=label_encoder(wine) #appying the function for label encoder


#------------------------------------------------------------------------------------------------------------------------------#
X=wine.drop('quality',axis=1)  #storing the X features 
y=wine['quality'] # defining the target variable


from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

X_covariance_matrix = np.cov(X_train.T) #calculating the covariance for Transformed X matrix 

eig_vals, eig_vecs = np.linalg.eig(X_covariance_matrix) #calculating the eigenvales and eigen vectors

# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort(key=lambda x: x[0], reverse=True)


tot = sum(eig_vals) #calculating the sum of eigen values
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)] #soring the eigenvalues and calculating the variance explained
cum_var_exp = np.cumsum(var_exp) #calculating the cumulative sum  of variance explained 
print ("Variance captured by each component is \n",var_exp)
print(40 * '-')
print ("Cumulative variance captured as we travel each component \n",cum_var_exp)
print('\n')


value=min(cum_var_exp, key=lambda x:abs(x-95)) # identifing the neareast value of 95 in the list 

n_components=list(cum_var_exp).index(value) # getting that nearest value of 95 and storing the index of that value in list
#as no.of components


from numpy import linalg as LA # importing the LA
kernel=['linear','poly','rbf', 'sigmoid', 'cosine']  #giving the list of kernels in Kernelpca
reconstruction_error=[] #empty list of reconstruction error
for i in  kernel:
    if kernel == ['poly','rbf', 'sigmoid']: #if these kernel's are comes then this gamma value has to choose 
        gamma=1 / (len(X.columns) * X.values.var())
    else:
        gamma=1/len(X.columns) #else the gamma values will be this 
    pca = KernelPCA(n_components=n_components,kernel=i,gamma=gamma,fit_inverse_transform=True)# deffing the kernelpca and 
    #the important parameters 
    pca2_results = pca.fit_transform(X_train) #fitting kernel pca 
    pca2_proj_back=pca.inverse_transform(pca2_results) #performing the inverse transform on the transformed fetaures with kernelpca
    reconstruction_error.append(LA.norm((X_train, pca2_proj_back),None)) #calculating the difference between the orginal 
    #feature and inverse transformed features of kernel pca
    
print('reconstruction_error',reconstruction_error) #printing the reconstruction error
print('\n')


#------------------------------------------------------------------------------------------------------------------------------#
X=wine.drop('quality',axis=1)  #storing the X features 
y=wine['quality'] # defining the target variable


from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='linear',gamma=1/len(X.columns)) # defing the kernelpca parameters
# checking kernel parameter with linear and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestRegressor  #importing the randomforest

classifier= RandomForestRegressor() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 


from sklearn.metrics import r2_score  #importing the classification report
print('KernelPca Linear Model Results','\n',r2_score(y_test, y_pred))#printing the classification report
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=wine.drop('quality',axis=1)
y=wine['quality']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='poly',gamma=1 / (len(X.columns) * X.values.var()))# defing the kernelpca parameters
# checking kernel parameter with poly and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestRegressor  #importing the randomforest

classifier= RandomForestRegressor() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import r2_score
print('KernelPca poly Model Results','\n',r2_score(y_test, y_pred))
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=wine.drop('quality',axis=1)
y=wine['quality']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='rbf',gamma=1 / (len(X.columns) * X.values.var())) # defing the kernelpca parameters
# checking kernel parameter with rbf and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestRegressor  #importing the randomforest

classifier= RandomForestRegressor() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import r2_score
print('KernelPca rbf Model Results','\n',r2_score(y_test, y_pred))
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=wine.drop('quality',axis=1)
y=wine['quality']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='sigmoid',gamma=1 / (len(X.columns) * X.values.var()))# defing the kernelpca parameters
# checking kernel parameter with sigmoid and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestRegressor  #importing the randomforest

classifier= RandomForestRegressor() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import r2_score
print('KernelPca sigmoid Model Results','\n',r2_score(y_test, y_pred))
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=wine.drop('quality',axis=1)
y=wine['quality']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='cosine',gamma=1 / (len(X.columns)))# defing the kernelpca parameters
# checking kernel parameter with cosine and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestRegressor  #importing the randomforest

classifier= RandomForestRegressor() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 


from sklearn.metrics import r2_score
print('KernelPca cosine Model Results','\n',r2_score(y_test, y_pred))

Variance captured by each component is 
 [29.29321718754935, 14.320363009824618, 11.106103089574539, 9.259294133372196, 8.84849617879475, 8.534013740313142, 6.605436572268536, 5.448713451289449, 3.7649424158637173, 2.6317012597137124, 0.18771896143598701]
----------------------------------------
Cumulative variance captured as we travel each component 
 [ 29.29321719  43.6135802   54.71968329  63.97897742  72.8274736
  81.36148734  87.96692391  93.41563736  97.18057978  99.81228104
 100.        ]


reconstruction_error [318.2119153495568, 306.2982716548172, 295.49157949581024, 303.8991561690859, 308.3608730147102]


KernelPca Linear Model Results 
 0.4823779543243877


KernelPca poly Model Results 
 0.49186766134513915


KernelPca rbf Model Results 
 0.4868597204251912


KernelPca sigmoid Model Results 
 0.4903269782274393


KernelPca cosine Model Results 
 0.47162533763926207


# House

In [143]:
house=null_replace(house)
house=label_encoder(house)

#------------------------------------------------------------------------------------------------------------------------------#
X=house.drop('sqft_lot15',axis=1)
y=house['sqft_lot15']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

X_covariance_matrix = np.cov(X_train.T) #calculating the covariance for Transformed X matrix 

eig_vals, eig_vecs = np.linalg.eig(X_covariance_matrix) #calculating the eigenvales and eigen vectors

# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort(key=lambda x: x[0], reverse=True)


tot = sum(eig_vals) #calculating the sum of eigen values
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)] #soring the eigenvalues and calculating the variance explained
cum_var_exp = np.cumsum(var_exp) #calculating the cumulative sum  of variance explained 
print ("Variance captured by each component is \n",var_exp)
print(40 * '-')
print ("Cumulative variance captured as we travel each component \n",cum_var_exp)
print('\n')


value=min(cum_var_exp, key=lambda x:abs(x-95)) # identifing the neareast value of 95 in the list 

n_components=list(cum_var_exp).index(value) # getting that nearest value of 95 and storing the index of that value in list
#as no.of components


from numpy import linalg as LA # importing the LA
kernel=['linear','poly','rbf', 'sigmoid', 'cosine']  #giving the list of kernels in Kernelpca
reconstruction_error=[] #empty list of reconstruction error
for i in  kernel:
    if kernel == ['poly','rbf', 'sigmoid']: #if these kernel's are comes then this gamma value has to choose 
        gamma=1 / (len(X.columns) * X.values.var())
    else:
        gamma=1/len(X.columns) #else the gamma values will be this 
    pca = KernelPCA(n_components=n_components,kernel=i,gamma=gamma,fit_inverse_transform=True)# deffing the kernelpca and 
    #the important parameters 
    pca2_results = pca.fit_transform(X_train) #fitting kernel pca 
    pca2_proj_back=pca.inverse_transform(pca2_results) #performing the inverse transform on the transformed fetaures with kernelpca
    reconstruction_error.append(LA.norm((X_train, pca2_proj_back),None)) #calculating the difference between the orginal 
    #feature and inverse transformed features of kernel pca
    
print('reconstruction_error',reconstruction_error) #printing the reconstruction error
print('\n')



#------------------------------------------------------------------------------------------------------------------------------#
X=house.drop('sqft_lot15',axis=1)
y=house['sqft_lot15']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='linear',gamma=1/len(X.columns)) # defing the kernelpca parameters
# checking kernel parameter with linear and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestRegressor  #importing the randomforest

classifier= RandomForestRegressor() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 


from sklearn.metrics import r2_score  #importing the classification report
print('KernelPca Linear Model Results','\n',r2_score(y_test, y_pred))#printing the classification report
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=house.drop('sqft_lot15',axis=1)
y=house['sqft_lot15']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='poly',gamma=1 / (len(X.columns) * X.values.var()))# defing the kernelpca parameters
# checking kernel parameter with poly and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestRegressor  #importing the randomforest

classifier= RandomForestRegressor() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import r2_score
print('KernelPca poly Model Results','\n',r2_score(y_test, y_pred))
print('\n')


#------------------------------------------------------------------------------------------------------------------------------#
X=house.drop('sqft_lot15',axis=1)
y=house['sqft_lot15']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='rbf',gamma=1 / (len(X.columns) * X.values.var())) # defing the kernelpca parameters
# checking kernel parameter with rbf and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestRegressor  #importing the randomforest

classifier= RandomForestRegressor() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import r2_score
print('KernelPca rbf Model Results','\n',r2_score(y_test, y_pred))
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=house.drop('sqft_lot15',axis=1)
y=house['sqft_lot15']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='sigmoid',gamma=1 / (len(X.columns) * X.values.var()))# defing the kernelpca parameters
# checking kernel parameter with sigmoid and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestRegressor  #importing the randomforest

classifier= RandomForestRegressor() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import r2_score
print('KernelPca sigmoid Model Results','\n',r2_score(y_test, y_pred))
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=house.drop('sqft_lot15',axis=1)
y=house['sqft_lot15']
from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='cosine',gamma=1 / (len(X.columns)))# defing the kernelpca parameters
# checking kernel parameter with cosine and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestRegressor  #importing the randomforest

classifier= RandomForestRegressor() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 


from sklearn.metrics import r2_score
print('KernelPca cosine Model Results','\n',r2_score(y_test, y_pred))

Variance captured by each component is 
 [31.655100740264388, 12.248609345935519, 7.81593763856578, 7.153855385437388, 6.088862294193866, 5.511587674422788, 4.7853504350649505, 4.223966417817956, 3.8853622380728536, 3.256449811660221, 2.951799419841878, 2.4369512527287784, 2.1206225004036168, 1.6215300438108184, 1.5037532593074445, 1.1086680563537676, 0.9058499749916994, 0.7257435111262726, 8.314614143677452e-16]
----------------------------------------
Cumulative variance captured as we travel each component 
 [ 31.65510074  43.90371009  51.71964772  58.87350311  64.9623654
  70.47395308  75.25930351  79.48326993  83.36863217  86.62508198
  89.5768814   92.01383265  94.13445515  95.7559852   97.25973846
  98.36840651  99.27425649 100.         100.        ]


reconstruction_error [191.89243752823918, 182.45269569687068, 166.02585661172063, 153.69587537532584, 185.34911150559557]


KernelPca Linear Model Results 
 0.4954749648705049


KernelPca poly Model Results 
 -0.00907653180427559


# Medical Insurance

In [8]:
insurance=null_replace(insurance)
insurance=label_encoder(insurance)

#------------------------------------------------------------------------------------------------------------------------------#
X=insurance.drop('charges',axis=1)
y=insurance['charges']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

X_covariance_matrix = np.cov(X_train.T) #calculating the covariance for Transformed X matrix 

eig_vals, eig_vecs = np.linalg.eig(X_covariance_matrix) #calculating the eigenvales and eigen vectors

# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort(key=lambda x: x[0], reverse=True)


tot = sum(eig_vals) #calculating the sum of eigen values
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)] #soring the eigenvalues and calculating the variance explained
cum_var_exp = np.cumsum(var_exp) #calculating the cumulative sum  of variance explained 
print ("Variance captured by each component is \n",var_exp)
print(40 * '-')
print ("Cumulative variance captured as we travel each component \n",cum_var_exp)
print('\n')


value=min(cum_var_exp, key=lambda x:abs(x-95)) # identifing the neareast value of 95 in the list 

n_components=list(cum_var_exp).index(value) # getting that nearest value of 95 and storing the index of that value in list
#as no.of components


from numpy import linalg as LA # importing the LA
kernel=['linear','poly','rbf', 'sigmoid', 'cosine']  #giving the list of kernels in Kernelpca
reconstruction_error=[] #empty list of reconstruction error
for i in  kernel:
    if kernel == ['poly','rbf', 'sigmoid']: #if these kernel's are comes then this gamma value has to choose 
        gamma=1 / (len(X.columns) * X.values.var())
    else:
        gamma=1/len(X.columns) #else the gamma values will be this 
    pca = KernelPCA(n_components=n_components,kernel=i,gamma=gamma,fit_inverse_transform=True)# deffing the kernelpca and 
    #the important parameters 
    pca2_results = pca.fit_transform(X_train) #fitting kernel pca 
    pca2_proj_back=pca.inverse_transform(pca2_results) #performing the inverse transform on the transformed fetaures with kernelpca
    reconstruction_error.append(LA.norm((X_train, pca2_proj_back),None)) #calculating the difference between the orginal 
    #feature and inverse transformed features of kernel pca
    
print('reconstruction_error',reconstruction_error) #printing the reconstruction error
print('\n')



#------------------------------------------------------------------------------------------------------------------------------#
X=insurance.drop('charges',axis=1)
y=insurance['charges']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='linear',gamma=1/len(X.columns)) # defing the kernelpca parameters
# checking kernel parameter with linear and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestRegressor  #importing the randomforest

classifier= RandomForestRegressor() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 


from sklearn.metrics import r2_score  #importing the classification report
print('KernelPca Linear Model Results','\n',r2_score(y_test, y_pred))#printing the classification report
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=insurance.drop('charges',axis=1)
y=insurance['charges']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='poly',gamma=1 / (len(X.columns) * X.values.var()))# defing the kernelpca parameters
# checking kernel parameter with poly and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestRegressor  #importing the randomforest

classifier= RandomForestRegressor() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import r2_score
print('KernelPca poly Model Results','\n',r2_score(y_test, y_pred))
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=insurance.drop('charges',axis=1)
y=insurance['charges']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='rbf',gamma=1 / (len(X.columns) * X.values.var())) # defing the kernelpca parameters
# checking kernel parameter with rbf and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestRegressor  #importing the randomforest

classifier= RandomForestRegressor() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import r2_score
print('KernelPca rbf Model Results','\n',r2_score(y_test, y_pred))
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=insurance.drop('charges',axis=1)
y=insurance['charges']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='sigmoid',gamma=1 / (len(X.columns) * X.values.var()))# defing the kernelpca parameters
# checking kernel parameter with sigmoid and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestRegressor  #importing the randomforest

classifier= RandomForestRegressor() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 

from sklearn.metrics import r2_score
print('KernelPca sigmoid Model Results','\n',r2_score(y_test, y_pred))
print('\n')

#------------------------------------------------------------------------------------------------------------------------------#
X=insurance.drop('charges',axis=1)
y=insurance['charges']

from sklearn.preprocessing import StandardScaler #importing the Standardscaler
sc = StandardScaler() # giving variable to Standardscaler 
X_train = sc.fit_transform(X) #fitting and transform standard scaler 

pca = KernelPCA(n_components=n_components,kernel='cosine',gamma=1 / (len(X.columns)))# defing the kernelpca parameters
# checking kernel parameter with cosine and respective gamma value
pca2_results = pca.fit_transform(X_train)

columns=[] #empty list for columns from 
for i in range(n_components): #for loop n components
    columns.append('KPCA'+str(i)) #appending the columns to the empty list
df=pd.DataFrame(pca2_results,columns=columns) #creating the dataframe for kernelpca components
df['target']=y #adding the target variable for the dataframe


X=df.drop('target',axis=1) #defing the X features 
y=df['target'] # defining the target variable

from sklearn.model_selection import train_test_split #importing the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42) 

from sklearn.ensemble import RandomForestRegressor  #importing the randomforest

classifier= RandomForestRegressor() # giving the variable to randomforest
classifier.fit(X_train, y_train) #fitting X and y 
y_pred= classifier.predict(X_test) #predicting the X test 


from sklearn.metrics import r2_score
print('KernelPca cosine Model Results','\n',r2_score(y_test, y_pred))

Variance captured by each component is 
 [20.060309157445776, 18.136640229738703, 16.992313681861653, 16.087702325770536, 15.399424412600704, 13.323610192582619]
----------------------------------------
Cumulative variance captured as we travel each component 
 [ 20.06030916  38.19694939  55.18926307  71.27696539  86.67638981
 100.        ]


reconstruction_error [122.37806762289244, 122.98626428770282, 116.95900766908521, 114.86088234283275, 120.64109409880736]


KernelPca Linear Model Results 
 0.8100594308937372


KernelPca poly Model Results 
 0.818338631774253


KernelPca rbf Model Results 
 0.8049215756105762


KernelPca sigmoid Model Results 
 0.8077429084708132


KernelPca cosine Model Results 
 0.813439273423118
